In [ ]:
from time import sleep
from datetime import datetime
from tradingview_screener import Query, col
# Set display options to prevent column wrapping
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0) # Adjust as needed, 0 will use the full available width

from colorama import Fore, Back, Style     
# symbol ='AAPL'   
# print(Fore.RED + 'some red text')
# print(  'and with a green background')
# print(Style.DIM + 'and in dim text')

# print(Back.GREEN + Style.DIM +f"Alert for {symbol}:")
# print(Style.RESET_ALL)

from collections import defaultdict
from tvDatafeed import TvDatafeed,Interval
tv = TvDatafeed()
support_line = defaultdict(float)
resistance_line = defaultdict(float)
import csv

def add_string_to_csv(filename, symbol):
    """
    Adds a string to a CSV file, writing each character as a separate field in a new row.

    Args:
        filename (str): The name of the CSV file to write to.
        string_data (str): The string to add to the CSV file.
    """
    with open(filename, 'a', newline='\n') as csvfile:
        current_time = datetime.now().strftime('%H:%M:%S')
        writer = csv.writer(csvfile)
        # strData = f":Alert for {symbol} ::  'https://in.tradingview.com/chart/N8zfIJVK/?symbol={symbol}' : Support Line: {support_line[symbol]}, Resistance Line: {resistance_line[symbol]}"
        writer.writerow([current_time,symbol,f'https://in.tradingview.com/chart/N8zfIJVK/?symbol={symbol}',])

def check_conditions(symbol, minute_data,length):
    if len(minute_data) < length:
        return
    avgV = minute_data['volume'].rolling(window=length).mean().iloc[-1]
    std = minute_data['volume'].rolling(window=length).std().iloc[-1]
    last_volume = minute_data['volume'].iloc[-1]
    last_close = minute_data['close'].iloc[-1]
    last_open = minute_data['open'].iloc[-length]

    condition = (last_volume - avgV > 2*std) or ((last_volume + minute_data['volume'].iloc[0]) / 2 - avgV > 2*std)

    if condition:
        support_line[symbol] = min(minute_data['low'].iloc[-length:]) if last_close > last_open else support_line[symbol]
        resistance_line[symbol] = max(minute_data['high'].iloc[-length:]) if last_close < last_open else resistance_line[symbol]
        print(Back.GREEN + Style.BRIGHT+Fore.BLUE +f"Alert for {symbol}: Support Line: {support_line[symbol]}, Resistance Line: {resistance_line[symbol]}")
        
        add_string_to_csv('SCREENER ALERTS_'+str(current_DATE)+'.csv',symbol)
        return True
    else :
        return False


master_combined =  pd.DataFrame()

import pandas as pd
import os

def append_df_to_csv(df, csv_path):
    if not os.path.exists(csv_path):
        df.to_csv(csv_path, mode='a', header=True, index=False)
    else:
        df.to_csv(csv_path, mode='a', header=False, index=False)


writeHeader=True
while True : 
  
 
    tvQuery = Query().set_markets('india') 
    _,dfNew = (tvQuery
     .select('name', 'close', 'volume', 'volume|1','volume|5','relative_volume_10d_calc|1', 'relative_volume_10d_calc|5','relative_volume_intraday|5','average_volume_60d_calc|1','average_volume_60d_calc|5')#'relative_volume_10d_calc','volume_change|1','relative_volume_intraday|5','relative_volume','VWMA','premarket_change','premarket_change_abs','premarket_change_from_open','premarket_change_from_open_abs','premarket_close','premarket_gap','premarket_high','premarket_low','premarket_open','premarket_volume')
    .where(      #col('relative_volume_10d_calc') > 1,
            col('relative_volume_intraday|5') > 1.2,
            # col('volume_change|5') > 0.2,
            col('volume')> col('average_volume_60d_calc|5'),
            col('volume|1')>10000,
            col('close') > 100,
            col('volume|1')  < col('volume') 
        )
    # .order_by('relative_volume_intraday|5', ascending=False)
    .order_by('volume|1', ascending=False).get_scanner_data()) 
   
    if not dfNew.empty :
        current_time = datetime.now().strftime('%H:%M:%S')
        
        current_DATE = datetime.now().strftime('%d_%m_%y')

        # dfNew.insert(2, 'baseURL', "https://in.tradingview.com/chart/N8zfIJVK/?symbol="+str(dfNew['ticker'])+"") 
        # Insert the current time as a new column at the beginning of the DataFrame
        dfNew.insert(3, 'URL',"") 
        dfNew['URL'] = "https://in.tradingview.com/chart/N8zfIJVK/?symbol="+dfNew['ticker']+" "

        dfNew.insert(0, 'current_timestamp', current_time)
        # Filter rows where col1 is 3 times col2
        # 
        filtered_df = dfNew[dfNew['volume|5'] > 2 * dfNew['average_volume_60d_calc|5']]
        if not filtered_df.empty :
            # dfNew['dateTime'] = current_time
            # filtered_df.to_csv('TimeBasedAlerts_VolSpike_'+str(current_DATE)+'.csv', mode='a', index=False, header=False)
            append_df_to_csv(filtered_df, 'SCREENER TimeBasedAlerts_VolSpike_'+str(current_DATE)+'.csv')
            print(f"=="*30)
            # print(filtered_df)
            writeHeader=False 
            # print(filtered_df)
            for row in filtered_df['ticker']:
                 
                print(row)
                splt = str(row).split(":")
                try : 
                    nifty_data=tv.get_hist(splt[1],splt[0],interval=Interval.in_5_minute,n_bars=100)
                    check_conditions(row,nifty_data,48)             
                        
                except  :
                    print(f" error for {row}")
           
                    pass
                
    sleep(15)
 